Things to do: 
1. API access to Kaggle
2. Yolo V8


In [9]:
import kagglehub
import os
import cv2
import pathlib
from ultralytics import YOLO
from PIL import Image
import yaml
import time

In [10]:
# THIS IS THE PATH: DO NOT CHANGE
path = kagglehub.dataset_download("rupankarmajumdar/crop-pests-dataset")

In [11]:
data_yaml_path = pathlib.Path(path) / "data.yaml"
data_cfg = {
    "path": str(path),
    "train": "train/images",
    "val":   "valid/images",  
    "test":  "test/images",
    "names": [
        "ant","bee","beetle","caterpillar","earthworm","earwig",
        "grasshopper","moth","slug","snail","wasp","weevil"
    ]
}
with open(data_yaml_path, "w") as f:
    yaml.safe_dump(data_cfg, f)
print("Wrote:", data_yaml_path)

Wrote: /Users/TanA20/.cache/kagglehub/datasets/rupankarmajumdar/crop-pests-dataset/versions/2/data.yaml


In [ ]:
# train the model on training data
model = YOLO("yolov8n.pt")  

start = time.time()
train_res = model.train(
    data=str(data_yaml_path),
    epochs=2,          
    imgsz=320,         
    batch=4,           
    device="mps",      
    workers=8,
    plots=False,       
    project="quick_test",
    name="yolo_sanity"
)
end = time.time()
train_time = end - start

New https://pypi.org/project/ultralytics/8.3.225 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.224 🚀 Python-3.10.18 torch-2.9.0 MPS (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/TanA20/.cache/kagglehub/datasets/rupankarmajumdar/crop-pests-dataset/versions/2/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scal

In [ ]:
# fit the model and use to predict test data
start = time.time()
test_res = model.val(
    data=str(data_yaml_path),
    split='test',
    imgsz=320,     # Use the same image size as training
    batch=4,
    device="mps",
    workers=4,
    plots=False
)
end = time.time()
test_time = end - start

Ultralytics 8.3.224 🚀 Python-3.10.18 torch-2.9.0 MPS (Apple M4 Pro)
Model summary (fused): 72 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.2±0.2 ms, read: 182.8±67.0 MB/s, size: 39.0 KB)
val: Scanning /Users/TanA20/.cache/kagglehub/datasets/rupankarmajumdar/crop-pests-dataset/versions/2/test/labels... 546 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 546/546 2.3Kit/s 0.2s0.0s
val: New cache created: /Users/TanA20/.cache/kagglehub/datasets/rupankarmajumdar/crop-pests-dataset/versions/2/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 137/137 1.7it/s 1:230.1sss
                   all        546        689      0.515      0.526      0.533      0.295
                   ant         54         87      0.474      0.356      0.351      0.145
                   bee         40         44      0.543      0.705      0.693      0.266
                beetle         41        

In [16]:
# metrics and testing

# Mean Average Precision (mAP)
mAP50 = test_res.results_dict['metrics/mAP50(B)']
mAP50_95 = test_res.results_dict['metrics/mAP50-95(B)']

# Precision, Recall, and F1-score (typically averaged across classes/batches)
precision = test_res.results_dict['metrics/precision(B)']
recall = test_res.results_dict['metrics/recall(B)']
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Note: YOLO does not directly output a single 'Accuracy' metric in the classification sense,
# nor a single 'Area Under the Curve (AUC)' value; mAP is the primary AUC equivalent.

print(f"Mean Average Precision (mAP@0.50): {mAP50:.4f}")
print(f"Mean Average Precision (mAP@0.50-0.95): {mAP50_95:.4f}")
print(f"Precision (Box): {precision:.4f}")
print(f"Recall (Box): {recall:.4f}")
print(f"F1-Score (Derived): {f1_score:.4f}\n")

def format_time(seconds):
    """Converts total seconds into minutes and format."""
    mins, secs = divmod(seconds, 60)
    return f"{int(mins):0d}m {secs:.2f}s"

print(f"Training Time (Total): {format_time(train_time)}")
print(f"Testing/Validation Time (Total): {format_time(test_time)}")

Mean Average Precision (mAP@0.50): 0.5331
Mean Average Precision (mAP@0.50-0.95): 0.2950
Precision (Box): 0.5153
Recall (Box): 0.5265
F1-Score (Derived): 0.5208

Training Time (Total): 33m 6.51s
Testing/Validation Time (Total): 1m 23.54s
